デフォルト

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# CSVファイルを読み込む
df = pd.read_csv("input/blogtext.csv")

# 必要な特徴量とターゲットを選択する
X_text = df['text']
y = df['gender']

# テキストデータから特徴量を抽出する
vectorizer = CountVectorizer()
X_features = vectorizer.fit_transform(X_text)

# 特徴量とターゲットを訓練データとテストデータに分割する
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

# ロジスティック回帰モデルをトレーニングする
model = LogisticRegression()
model.fit(X_train, y_train)

# テストデータで予測を行う
y_pred = model.predict(X_test)

# 予測精度を評価する
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


c:\Users\yujin\Anaconda3\envs\authorship\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6778147177759675


一人称"i"と"I"の数を特徴量として追加

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy import sparse

# CSVファイルを読み込む
df = pd.read_csv("input/blogtext.csv")

# 一人称表現の特徴量を追加する
df['count_i'] = df['text'].apply(lambda text: text.count('i'))
df['count_I'] = df['text'].apply(lambda text: text.count('I'))

# 必要な特徴量とターゲットを選択する
X_features = df[['count_i', 'count_I']]
y = df['gender']

# ターゲット変数のエンコード
gender_mapping = {'male': 0, 'female': 1}
y_encoded = y.map(gender_mapping)

# 特徴量とターゲットを訓練データとテストデータに分割する
X_train, X_test, y_train, y_test = train_test_split(X_features, y_encoded, test_size=0.2, random_state=42)

# ロジスティック回帰モデル1をトレーニングする
model1 = LogisticRegression()
model1.fit(X_train, y_train)

# テストデータで予測を行う
y_pred1 = model1.predict(X_test)

# テキストデータから特徴量を抽出する
vectorizer = CountVectorizer()
X_text = df['text']  # テキストデータの列
X_features_text = vectorizer.fit_transform(X_text)

# 特徴量とターゲットを訓練データとテストデータに分割する
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(X_features_text, y_encoded, test_size=0.2, random_state=42)

# ロジスティック回帰モデル2をトレーニングする
model2 = LogisticRegression()
model2.fit(X_train_text, y_train_text)

# テストデータで予測を行う
y_pred2 = model2.predict(X_test_text)

# 予測結果を結合する
X_stacked = pd.concat([pd.Series(y_pred1), pd.Series(y_pred2)], axis=1)

# スタッキングモデルのトレーニング用にデータを整形する
X_stacked_train, X_stacked_test, y_stacked_train, y_stacked_test = train_test_split(X_stacked, y_test_text, test_size=0.2, random_state=42)

# 最終的なスタッキングモデルをトレーニングする
stacking_model = LogisticRegression()
stacking_model.fit(X_stacked_train, y_stacked_train)

# スタッキングモデルで予測を行う
y_pred_stacked = stacking_model.predict(X_stacked_test)

# 予測精度を評価する
accuracy_stacked = accuracy_score(y_stacked_test, y_pred_stacked)
print(f'Stacked Model Accuracy: {accuracy_stacked}')

c:\Users\yujin\Anaconda3\envs\authorship\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Stacked Model Accuracy: 0.6789226478790548
